In [2]:
import string
import pandas as pd
import fitz
path = "C:/DAEN690/1A13.pdf"
pdf = fitz.open(path)
pdf.metadata

{'format': 'PDF 1.5',
 'title': 'DEPARTMENT OF TRANSPORTATION',
 'author': 'Susan Holshouser',
 'subject': '',
 'keywords': '',
 'creator': 'Acrobat PDFMaker 10.1 for Word',
 'producer': 'Adobe PDF Library 10.0',
 'creationDate': "D:20131212122723-06'00'",
 'modDate': "D:20131212122728-06'00'",
 'trapped': '',
 'encryption': None}

In [6]:
#add page to all words
def add_page(in_put, value):
    inn = []
    out = [[]]
    index = 0
#    v = str(value+1)
    for i in range(0, len(in_put)):
        if(i < len(in_put)-1):
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn
            index += 1
            inn = []
            out += [[]]
        else:
            inn += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            inn.insert( 8, value+1)
            out[index] = inn

    return out

#get all text from given pdf file
def get_all_text(data):
    pc = data.page_count
    word_box = []
    for i in range(0,pc):
        ct = data.load_page(i)
        gw = ct.get_text('words')
        # If the df do not need page anymore, #agw = ... and make wor_box += gw
        agw = add_page(gw, i)
        word_box += agw
    
    return word_box

# merge the words by block_no and line_no
def word_line(in_put):
    index = 0
    word_list = []
    output = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][7]+1 == in_put[i+1][7]):
            word_list += [in_put[i][4]]  
        else:
            if(in_put[i+1][7]==0):
                word_list += [in_put[i][4]]
                word_list = [' '.join(word_list)]
                word_list += [in_put[i][5], in_put[i][6], in_put[i][7],in_put[i][8]]
                word_list.insert( 0, in_put[i-in_put[i][7]][0])
                word_list.insert( 1, in_put[i-in_put[i][7]][1])
                word_list.insert( 2, in_put[i][2])
                word_list.insert( 3, in_put[i][3])
            #print(word_list)

            output[index] += word_list
            index += 1
            word_list = []
            output += [[]]
    output.pop(-1)
    return output



#get all text [x0, y0, x1, y1, word, block_no, line_no, word_no, page]
all_text = get_all_text(pdf)


# merge text by block_no & line_no
out = word_line(all_text)
#print(out)

# Get the X1 of Fuel
def get_fuel_x1(df):
    
    for i in range(0,len(df)):
        if(df[i][4].lower() == 'fuel'):
            return df[i][2]

fuel_x1 = get_fuel_x1(out)


# detect the word lines' class (key_term, header, info)
def detect_khi(in_put):
    index = 0
    temp_list= []
    out_put = [[]]
    for i in range(0,len(in_put)):
        ## old one add & (in_put[i][6] == 0) in both if & elif
        if(in_put[i][0] <= fuel_x1) & (in_put[i][7] < 5 ):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'key_term')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        elif(in_put[i][0] <= fuel_x1) & (in_put[i][4].find('Model') > 0):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'header')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        else:
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7], in_put[i][8]]
            temp_list.insert(8, 'info')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
    out_put.pop(-1)
    #print(out_put)
    return out_put

# Sometimes the header's number will be detect as key term, this function is used to solve it.
def hder(df):
    for i in range(1, len(df)-5):
        if(df[i+1][8] == "header") & (int(df[i][1]) == int(df[i+1][1])):
            df[i][8] = "header"
    return df


# use to catch data in a single page
def gp(in_put):
    out = []
    for i in range(0,len(in_put)-8):
        if(in_put[i][8] == 1):
            out += [in_put[i]]
    return out

# merge the lines that have same y0 in same page 

def merge_info_same_y0(df):
    fill_lst = [0,0,0,0,0,0,0,0,0,0]
    df += [[]]
    df[-1] = fill_lst
    num = 0
    index = 0
    lst = []
    out = [[]]
    
    for i in range(0 ,len(df)-1):
        if(df[i][8] == 'info') & (df[i+1][8] == 'info') & (round(df[i][1]) == round(df[i+1][1])):
            lst += [df[i][4]]
            num += 1
        else:
            lst += [df[i][4]]
            lst = ['  '.join(lst)]
            lst += [df[i][5], df[i][6], df[i][7], df[i][8], df[i][9]]
            lst.insert( 0, df[i-num][0])
            lst.insert( 1, df[i][1])
            lst.insert( 2, df[i][2])
            lst.insert( 3, df[i][3])
            
            out[index] = lst
            index += 1
            lst = []
            out += [[]]
            num = 0
    out.pop(-1)
    return out

# merge the lines that have same x0 in same page, and the selected lines must in the right side 
def merge_all_info(df):
    fill_lst = [0,0,0,0,0,0,0,0,0,0]
    df += [[]]
    df[-1] = fill_lst
    num = 0
    index = 0
    lst = []
    out = [[]]
    for i in range(0 ,len(df)-1):
        if(df[i][8] == 'info') & (df[i+1][8] == 'info') & (int(df[i][0]) == int(df[i+1][0])) & (df[i][0] > 106):
            lst += [df[i][4]]
            num += 1
        else:
            lst += [df[i][4]]
            lst = ['  '.join(lst)]
            lst += [df[i][5], df[i][6], df[i][7], df[i][8], df[i][9]]
            lst.insert( 0, df[i-num][0])
            lst.insert( 1, df[i][1])
            lst.insert( 2, df[i][2])
            lst.insert( 3, df[i][3])
            
            out[index] = lst
            index += 1
            lst = []
            out += [[]]
            num = 0
    out.pop(-1)
    return out    


# pre process the data to data frame and ready to convert to excel file
def prep_to_xlsx(df):
    index = 0
    lst = []
    out = [[]]
    for i in range(0, len(df)):
        if(df[i][0] < 106):
            lst = [df[i][4], '']
            out[index] = lst
            lst = []
            out += [[]]
            index += 1
        else:
            lst = ['', df[i][4]]
            out[index] = lst
            lst = []
            out += [[]]
            index += 1
    
    out.pop(-1)
    return out

In [136]:

#sorted key + lambda 
# ref: https://blog.csdn.net/w417950004/article/details/86253721
# reorder the word lines by coordinates (page > y0 > x0)
pall = sorted(out, key = lambda x:(int(x[8]), float(x[1]), float(x[0])))
#for i in range(0, len(pall)):
#    print(pall[i][8], pall[i][1], pall[i][0], pall[i][4])


# classify the word to (header , key_term, info), 
#the output will be [x0, y0, x1, y1, word, block_no, line_no, word_no, class, page]
khi = detect_khi(pall)
#print(khip1)
khi = hder(khi)

# drop  (cont’d)
for i in range(0, len(khi)-20):
    if(khi[i][4].find('(cont’d)') > 0):
        khi.pop(i)
        i = i-1


#drop punctuation
#https://datagy.io/python-remove-punctuation-from-string/
for i in range(0, len(khi)-1):
    if(khi[i][8] == 'key_term'):
        khi[i][4] = khi[i][4].translate(str.maketrans('', '', string.punctuation))
        print(khi[i][4])

# Use page1 to test the function
# Be used to select page, if need all df, ignore this 
#p1 = gp(out)
#for i in range(0,len(p1)):
#    print(p1[i][8])

infoSy0 = merge_info_same_y0(khi)
#for i in range(0, len(infoSy0)):
#    print(infoSy0[i][4])

info = merge_all_info(infoSy0)
#for i in range(0, len(info)):
#    print(info[i][4])



Type Certificate Holder
Type Certificate Holder Record
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Airspeed Limits CAS
CG Range
Page No
Rev No
1A13
I
Empty Wt CG Range
Datum
Maximum Weight
Minimum Crew
No of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Manufacturer’s Serial Numbers
Certification Basis
Equipment
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Airspeed Limits CAS
CG Range
Empty Wt CG Range
Datum
Maximum Weight
Minimum Crew
No of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Manufacturer’s Serial Numbers
Certification Basis
Equipment
Model LA4
Model LA4A
Model LA4P
Engines
Fuel
Engine Limits
Propeller and Propeller Limits
1A13
Airspeed Limits CAS
CG Range
Empty Wt CG Range
Datum
Maximum Weight
Minimum Crew
No of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Aileron Cable Rigging Tension
Manufacturer’s Serial Numbers
Certification Basis
Equipment
Engines
Fuel
Engin

In [84]:
for i in range(0 ,len(khi)-1):
    print(khi[i][4])

DEPARTMENT OF TRANSPORTATION
FEDERAL AVIATION ADMINISTRATION
1A13
Revision 28
Revo, Inc.
COLONIAL C-1
COLONIAL C-2
LAKE LA-4
LAKE LA-4A
LAKE LA-4P
LAKE LA-4-200
LAKE MODEL 250
December 12, 2013
TYPE CERTIFICATE DATA SHEET NO. 1A13
Type Certificate Holder
Revo, Incorporated
1396 Grandview Boulevard
Kissimmee, FL 34744
Type Certificate Holder Record
Colonial Aircraft Company transferred TC 1A13 to Lake Aircraft Corporation on
December 23, 1959.
Lake Aircraft Corporation transferred TC 1A13 to Consolidated Aeronautics, Inc. on
May 7, 1963.
Consolidated Aeronautics, Inc. transferred TC 1A13 to Revo, Incorporated on
October 29, 1986.
Revo, Incorporated transferred TC 1A13 to Global Amphibians, LLC on
October 28, 2002.
Global Amphibians, LLC transferred TC 1A13 to Revo, Incorporated on
September 30, 2004
I.
Colonial Model C-1, 3 PCAmM (Normal Category), Approved 19 September 1955
Engine
Lycoming O-320-A2A
Fuel
80/87 minimum grade aviation gasoline
Engine Limits
For all operations, 2700 r.p.m

In [137]:
def all_header(df):
    for i in range(1 , len(df)-3):
        if(df[i-1][8] == 'header') & (df[i][4] != 'Engine') or (df[i][4] != 'Engines'):
            df[i][8] = 'header'
    return df



In [138]:
ah = all_header(khi)
for i in range(0,len(ah)-1):
    if(ah[i][8] == 'header'):
        print(ah[i][4])

FEDERAL AVIATION ADMINISTRATION
1A13
Revision 28
Revo, Inc.
COLONIAL C-1
COLONIAL C-2
LAKE LA-4
LAKE LA-4A
LAKE LA-4P
LAKE LA-4-200
LAKE MODEL 250
December 12, 2013
TYPE CERTIFICATE DATA SHEET NO. 1A13
Type Certificate Holder
Revo, Incorporated
1396 Grandview Boulevard
Kissimmee, FL 34744
Type Certificate Holder Record
Colonial Aircraft Company transferred TC 1A13 to Lake Aircraft Corporation on
December 23, 1959.
Lake Aircraft Corporation transferred TC 1A13 to Consolidated Aeronautics, Inc. on
May 7, 1963.
Consolidated Aeronautics, Inc. transferred TC 1A13 to Revo, Incorporated on
October 29, 1986.
Revo, Incorporated transferred TC 1A13 to Global Amphibians, LLC on
October 28, 2002.
Global Amphibians, LLC transferred TC 1A13 to Revo, Incorporated on
September 30, 2004
I.
Colonial Model C-1, 3 PCAmM (Normal Category), Approved 19 September 1955
Engine
Lycoming O-320-A2A
Fuel
80/87 minimum grade aviation gasoline
Engine Limits
For all operations, 2700 r.p.m. (150 hp)
Propeller and Prop

In [85]:
def all_header(df):
    for i in range(1 , len(df)):
        if(df[i-1][8] == 'header') & (df[i+1][4].lower != 'engine')
    fill_lst = [0,0,0,0,0,0,0,0,0,0]
    df += [[]]
    df[-1] = fill_lst
    num = 0
    index = 0
    lst = []
    out = [[]]
    
    for i in range(0,len(df)-1):
        str(df[i][4])
        if(df[i][8] != 'header'):
            lst += [df[i][0], df[i][1], df[i][2], df[i][3], df[i][4], df[i][8], df[i][9]]
            out[index] = lst
            lst = []
            index += 1
            out += [[]]
            
        if(df[i][8] == 'header') & (num == 0) & (str(df[i+1][4]) != 'Engine'):
            lst += [df[i][4]]
            num += 1
            
        elif(len(lst) != 0) & (str(df[i+1][4]) != 'Engine'):
            lst += [df[i][4]]
            num += 1
            
        elif(str(df[i][4]).lower() != 'engine'):
            lst += [df[i][4]]
            lst = [' '.join(lst)]
            lst += [df[i][8], df[i][9]]
            lst.insert( 0, df[i-num][0])
            lst.insert( 1, df[i-num][1])
            lst.insert( 2, df[i-num][2])
            lst.insert( 3, df[i][3])            
            num = 0
            out[index] = lst
            lst = []
            index += 1
            out += [[]]

    
    return out

ah = all_header(khi)

TypeError: sequence item 0: expected str instance, int found

In [99]:
for i in range(0,len(ah)-1):
    if(ah[i][8] == 'header'):
        print(ah[i][4])

I.
Colonial Model C-1, 3 PCAmM (Normal Category), Approved 19 September 1955
II. Colonial Model C-2, 4 PCAmM (Normal Category), Approved 24 December 1957
(Same as Model C-1 except for engine installation, cockpit arrangement, horizontal tail surfaces, longitudinal trim system,
III. Lake Model LA-4, Approved 26 July 1960; Lake Model LA-4A, Approved 1 June 1960; Lake Model LA-4P,
IV. Lake Model LA-4-200; 4 PCAmM (Normal Category) Approved 26 May 1970
V. Lake Model 250; 4 PCAmM (Normal Category) Approved 30 June 1983; 6 PCAmM (Normal Category)
Lake Model 250, 4-place is the same as LA-4-200 except for engine installation, vertical and horizontal tail surface configuration,
Lake Model 250-6-place is the same as Model 250, 4-place except for rear seat, auxiliary fuel (wing float) tanks are optional,


In [97]:
for i in range(0 ,len(ah)-1):
    print(ah[i])

[226.6199951171875, 56.783992767333984, 392.6072082519531, 69.24899291992188, 'DEPARTMENT OF TRANSPORTATION', 3, 0, 2, 'info', 1]
[221.94000244140625, 67.34100341796875, 397.2421569824219, 79.80599975585938, 'FEDERAL AVIATION ADMINISTRATION', 3, 1, 2, 'info', 1]
[527.219970703125, 79.32299041748047, 547.199951171875, 91.32898712158203, '1A13', 4, 0, 0, 'info', 1]
[503.8800048828125, 89.88298797607422, 547.2059326171875, 101.88898468017578, 'Revision 28', 5, 0, 1, 'info', 1]
[509.82000732421875, 100.44298553466797, 547.1698608398438, 112.44898223876953, 'Revo, Inc.', 6, 0, 1, 'info', 1]
[459.0, 111.00298309326172, 520.739990234375, 123.00897979736328, 'COLONIAL C-1', 7, 1, 1, 'info', 1]
[459.0, 121.56298065185547, 520.739990234375, 133.56898498535156, 'COLONIAL C-2', 8, 1, 1, 'info', 1]
[459.0, 132.1229705810547, 504.7740173339844, 144.1289825439453, 'LAKE LA-4', 9, 1, 1, 'info', 1]
[459.0, 142.68296813964844, 511.2720031738281, 154.68898010253906, 'LAKE LA-4A', 10, 1, 1, 'info', 1]
[45

## px = prep_to_xlsx(info)

for i in range(0, len(px)-1):
    px[i][0].translate(str.maketrans('', '', string.punctuation))
#for i in range(0, len(px)):
#print(px)
df = pd.DataFrame(px, columns=['L', 'R'])
df

In [273]:
px1 = prexlsx(infoSy0)
df1 = pd.DataFrame(px1, columns=['L', 'R'])
df1

,L,R
0,,DEPARTMENT OF TRANSPORTATION
1,,FEDERAL AVIATION ADMINISTRATION
2,,1A13
3,,Revision 28
4,,"Revo, Inc."
...,...,...
492,NOTE 10.,
493,,"4-place version available for Lake Model 250, ..."
494,,"SA745NE approved April 19, 1990, issued to Rev..."
495,,"04073, now located at 1396 Grandview Blvd., Ki..."


In [236]:
# not done yet
#def xlsx(in_put):
#    lst = in_put
#    df = pd.DataFrame(in_put, columns=['L', 'R'])
#    df.to_excel("1A13_P1.xlsx", index = False)

In [238]:
# xlsx(px)